In [ ]:
# pog
# oh yea we pog
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import floor
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, StackingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.losses import CategoricalCrossentropy

folder = '/kaggle/input/stat-441-w2022-data-challenge/'

# We get the necessary data
train_data = pd.read_csv(folder+'train.csv') # training data
test_data = pd.read_csv(folder+'test.csv') # test data
sample_sub = pd.read_csv(folder+'sample_submission.csv')
train_data.sample(5)
# train_data.info(verbose=True, show_counts=True)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

To begin with this competition, we create a function that can provide us a training set and testing set depending on the dataframe. We can further change the size of the training and testing sets through the optional parameter.

In [ ]:
def train_test_sets(X,y,train_size=0.75):
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=train_size, random_state=441)
    return (X_train, X_test, y_train, y_test)

We will now clean the dataframe to get rid of null values. We will using two approaches for data cleaning. In the first approach, we will drop the columns if there are any columns with less than 50% non-NA values. In the second approach, we use 60%. We believe that using this approach can give an idea of how the sizes can affect the accuracy, and subsequently, entropy as well.  

In both the approaches, we will impute the remaining columns with their respective modes. In the appendix, we have explained more about the reason behind using "mode" for imputation.  

In [ ]:
train_data_clean_1 = train_data.dropna(axis = 1, thresh = 8698) # drop columns with 50% or more null values
train_data_clean_1.drop(['uniqueid','personid'], axis=1, inplace=True) # drop the unnecessary columns
for name in train_data_clean_1.columns:
    # impute the columns with mode
    train_data_clean_1.loc[train_data_clean_1[name].isnull(), name] = train_data_clean_1[name].mode()[0]
print(train_data_clean_1.shape)


# Logistic Regression approach  


In [ ]:
# X = train_data_clean_1.iloc[:,:-1]
# y = train_data_clean_1['health']
# X_train, X_test, y_train, y_test = train_test_sets(X,y, 0.8)

# params = {'penalty':['l2','elasticnet','none'], 'C': [0.1, 0.5, 1],
#          'warm_start':[True, False], 'l1_ratio':[0, 0.1, 0.5, 0.75, 1]}
# logmod = LogisticRegression(multi_class="multinomial", max_iter=1000, solver='saga')
# log = GridSearchCV(logmod, params, n_jobs=-1, verbose=0, refit='neg_log_loss', scoring=['accuracy','neg_log_loss'])
# log.fit(X_train, y_train)
# print(log.best_estimator_, log.best_score_, log.best_params_, sep='\n')
# log.score(X_test, y_test)


LogisticRegression(C=0.1, l1_ratio=0.1, max_iter=1000,
                   multi_class='multinomial', solver='saga', warm_start=True)  
                   
-1.3887890226274735  

{'C': 0.1, 'l1_ratio': 0.1, 'penalty': 'l2', 'warm_start': True}  

-1.3998282372992474

# XGB Classifier

In [ ]:
# X = train_data.iloc[:,3:-1]
# y = train_data['health']
# X_train, X_test, y_train, y_test = train_test_sets(X,y, 0.8)

# encoder = LabelEncoder()
# encoder.fit(y_train)
# ytrain = encoder.transform(y_train)

# encoder.fit(y_test)
# ytest = encoder.transform(y_test)

# params = {'n_estimators':[20, 50, 75, 100, 120], 'learning_rate': (0.1, 0.15, 0.2, 0.25, 0.3)}
# xgb = XGBClassifier(objective='multi:softprob', n_jobs=-1, eval_metric='mlogloss', \
#                     max_depth=2, verbosity=0, missing=np.nan)
# xgb = GridSearchCV(xgb, params, n_jobs=-1, verbose=0, refit='neg_log_loss', scoring=['accuracy','neg_log_loss'])
# xgb.fit(X_train, y_train)
# print(xgb.best_estimator_, xgb.best_score_, xgb.best_params_, sep='\n')
# xgb.score(X_test, y_test)

Best estimator is `XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.2, max_delta_step=0,
              max_depth=2, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=-1,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=0)`  
Scores and params are `-1.214446512794508 {'learning_rate': 0.2, 'n_estimators': 100}`

In [ ]:
# X = train_data_clean_1.iloc[:,:-1]
# y = train_data_clean_1['health']
# X_train, X_test, y_train, y_test = train_test_sets(X,y, 0.8)

# params = {'gamma': [0, 0.5, 1, 2, 5, 10, 20, 50], 'max_depth':[1, 2, 5, 10], 'eta':[0.1, 0.2, 0.5, 0.8, 1],\
#          'max_delta_step':[1, 2, 5, 10], 'subsample':[0.5, 0.75, 1], 'reg_lambda':[0, 0.2, 0.5, 0.75, 1],\
#          'reg_alpha':[0, 0.2, 0.5, 0.75, 1], 'tree_method':['exact','hist','gpu_hist','approx', 'auto']}
# xgb_model = XGBClassifier(objective='multi:softprob', verbosity=0, eval_metric='mlogloss')
# gs_cv = GridSearchCV(xgb_model, params, refit=False, scoring=['accuracy', 'neg_log_loss'], verbose=0, n_jobs=-1)

# gs_cv.fit(X_train, y_train)
# print(gs_cv.best_score_, gs_cv.best_estimator_)
# print(gs_cv.score(X_test, y_test))

# AdaBoost + XGBClassifer  

**Too slow hence should never be executed**

In [ ]:
# X = train_data_clean_1.iloc[:,:-1]
# y = train_data_clean_1['health']
# X_train, X_test, y_train, y_test = train_test_sets(X,y, 0.8)
# params = {'n_estimators':[1, 10, 20, 50, 100], 'learning_rate':[0.5, 1, 1.75, 2.5, 3]}
# adaModel = AdaBoostClassifier(XGBClassifier(objective='multi:softprob', eval_metric='mlogloss', max_depth=2),random_state=441)
# clf = GridSearchCV(adaModel, params, n_jobs=-1, verbose=0, scoring='accuracy')
# clf.fit(X_train, y_train)
# print(clf.best_params_, clf.best_score_)
# clf.score(X_test, y_test)

# AdaBoost

In [ ]:
# X = train_data_clean_1.iloc[:,:-1]
# y = train_data_clean_1['health']
# X_train, X_test, y_train, y_test = train_test_sets(X,y, 0.8)

# params = {'n_estimators':[1, 10, 20, 50, 100], 'learning_rate':[0.5, 1, 1.75, 2.5, 3]}
# ada_model = AdaBoostClassifier(random_state=441)
# ada = GridSearchCV(ada_model, params, n_jobs=-1, verbose=0, refit='neg_log_loss', scoring=['accuracy','neg_log_loss'])
# ada.fit(X_train, y_train)
# print(ada.best_estimator_, ada.best_score_, ada.best_params_)
# ada.score(X_test, y_test)

Best estimator is `AdaBoostClassifier(learning_rate=0.5, n_estimators=1, random_state=441)`  
Score and params are `1.3351385752744558 {'learning_rate': 0.5, 'n_estimators': 1}`

# DecisionTreeClassifier

In [ ]:
# X = train_data_clean_1.iloc[:,:-1]
# y = train_data_clean_1['health']
# X_train, X_test, y_train, y_test = train_test_sets(X,y, 0.8)

# params = {'max_depth':[1, 2, 5, 10, 15], 'min_samples_split': [2, 5, 10, 15, 20], 'min_impurity_decrease':[0.25, 0.5, 0.75]}
# dtc_model = DecisionTreeClassifier(criterion='entropy', random_state=441, max_features='auto')
# dtc = GridSearchCV(dtc_model, params, n_jobs=-1, verbose=0, refit='neg_log_loss', scoring=['accuracy','neg_log_loss'])
# dtc.fit(X_train, y_train)
# print(dtc.best_estimator_, dtc.best_score_, dtc.best_params_)
# dtc.score(X_test, y_test)

Best estimator is `DecisionTreeClassifier(criterion='entropy', max_depth=1, max_features='auto',
                       min_impurity_decrease=0.25, random_state=441)`  
Best score and params are `-1.3915510004920844 {'max_depth': 1, 'min_impurity_decrease': 0.25, 'min_samples_split': 2}`  

# AdaBoost + DecisionTree 

In [ ]:
# X = train_data_clean_1.iloc[:,:-1]
# y = train_data_clean_1['health']
# X_train, X_test, y_train, y_test = train_test_sets(X,y, 0.8)

# params = {'n_estimators':[1, 2, 5, 10], 'learning_rate':[0.1, 0.2, 0.5, 0.75]}
# ada_dtc_model = AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy', max_depth=1, max_features='auto',
#                        min_impurity_decrease=0.25, random_state=441),random_state=441)
# ada_dtc = GridSearchCV(ada_dtc_model, params, n_jobs=-1, verbose=0, refit='neg_log_loss', scoring=['accuracy','neg_log_loss'])
# ada_dtc.fit(X_train, y_train)
# print(ada_dtc.best_estimator_, ada_dtc.best_score_, ada_dtc.best_params_)
# ada_dtc.score(X_test, y_test)

Best estimator is `AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                 max_depth=1, max_features='auto', min_impurity_decrease=0.25, 
                 random_state=441), learning_rate=0.5, n_estimators=1, random_state=441)`  
Best scores and params are `-1.3915510004920844 {'learning_rate': 0.5, 'n_estimators': 1}`

# RandomForestClassifier

In [ ]:
# X = train_data_clean_1.iloc[:,:-1]
# y = train_data_clean_1['health']
# X_train, X_test, y_train, y_test = train_test_sets(X,y, 0.8)

# params = {'n_estimators':[250, 300], 'max_depth':[12, 15, 18],\
#          'min_samples_split':[0.01, 0],'bootstrap':[True, False]}
# rfc_model = RandomForestClassifier(criterion='entropy', max_features="auto", warm_start=True)
# rfc = GridSearchCV(rfc_model, params, n_jobs=-1, verbose=0, refit='neg_log_loss', scoring=['neg_log_loss','accuracy'])
# rfc.fit(X_train, y_train)
# print(rfc.best_params_, rfc.best_score_, rfc.best_estimator_, sep='\n')
# rfc.score(X_test, y_test)

Best estimator is `RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=15,
                       min_impurity_decrease=0, min_samples_split=0.01,
                       n_estimators=255, warm_start=True)`  
Best score and params are `
{'bootstrap': False, 'max_depth': 15, 'min_impurity_decrease': 0, 'min_samples_split': 0.01, 'n_estimators': 255, 'warm_start': True}
-1.2343588039175948`

# AdaBoost + RandomForest

In [ ]:
# X = train_data_clean_1.iloc[:,:-1]
# y = train_data_clean_1['health']
# X_train, X_test, y_train, y_test = train_test_sets(X,y, 0.8)

# params = {'n_estimators':[1, 10, 20, 50], 'learning_rate':[0.1, 0.2, 0.25, 0.5]}
# ada_rfc_model = AdaBoostClassifier(RandomForestClassifier(criterion='entropy', max_features="auto"), random_state=441)
# ada_rfc = GridSearchCV(ada_rfc_model, params, n_jobs=-1, verbose=0, scoring='neg_log_loss')
# ada_rfc.fit(X_train, y_train)
# print(ada_rfc.best_params_, ada_rfc.best_score_, ada_rfc.best_estimator_, sep='\n')
# ada_rfc.score(X_test, y_test)

Best estimator is `AdaBoostClassifier(base_estimator=RandomForestClassifier(criterion='entropy'),
                   learning_rate=0.1, n_estimators=1, random_state=441)`  
Best params and score are `{'learning_rate': 0.1, 'n_estimators': 1} -1.2836877005693468`


# Neural Network  

It's not worth it for this competition

In [ ]:
# X = train_data_clean_1.iloc[:,:-1]
# y = train_data_clean_1['health']
# X_train, X_test, y_train, y_test = train_test_sets(X,y, 0.8)
# X_keras = X_train.to_numpy()

# # encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(y_train)
# y_keras = encoder.transform(y_train)
# # convert integers to dummy variables (i.e. one hot encoded)
# y_keras = np_utils.to_categorical(y_keras)

# # es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patient=100, min_delta=1)

# neural_model = Sequential()
# # neural_model.add(Dropout(0.4, input_dim=180))
# neural_model.add(Dense(350, activation='tanh'))
# # neural_model.add(Dropout(0.2))
# neural_model.add(Dense(350, activation='softplus'))
# # neural_model.add(Dropout(0.2))
# neural_model.add(Dense(350, activation='exponential'))
# neural_model.add(Dense(5, activation='softmax'))
# neural_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # used the Mean Squared Error metric
# fitted = neural_model.fit(X_keras, y_keras, epochs=250, batch_size=1250, verbose=0) # fit the model
# print(f'{neural_model.evaluate(X_keras,y_keras, verbose=0)}') # Print the training MSE


In [ ]:
# print(f'{neural_model.evaluate(X_keras,y_keras, verbose=0)}') # Print the training MSE
# y_pred = neural_model.predict(X_test) # find predictions
# encoder.fit(y_test)
# # encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(y_test)
# y_keras_test = encoder.transform(y_test)
# y_keras_test = np_utils.to_categorical(y_keras_test)
# cce = CategoricalCrossentropy(from_logits=False)
# cce(y_keras_test[:,1:], y_pred).numpy()

# Stacking  

In [ ]:
X = train_data_cleanX = train_data_clean_1.iloc[:,:-1]
y = train_data_clean_1['health']
X_train, X_test, y_train, y_test = train_test_sets(X,y)

encoder = LabelEncoder()
encoder.fit(y_train)
ytrain = encoder.transform(y_train)
endoder = LabelEncoder()
encoder.fit(y_test)
ytest = encoder.transform(y_test)


rf = RandomForestClassifier(criterion='entropy', max_depth=18,
                        n_estimators=250, random_state=441, min_samples_split= 0.01,)
log = LogisticRegression(C=0.1, l1_ratio=0.1, max_iter=5000, penalty='elasticnet',
                   multi_class='multinomial', solver='saga', warm_start=True)
xgb = XGBClassifier(base_score=0.5, booster='gbtree',
              eval_metric='mlogloss', gamma=0, learning_rate=0.2,
              max_depth = 2, n_estimators=100, objective='multi:softprob',
              reg_alpha=0, reg_lambda=1, use_label_encoder=False, verbosity=0, random_state=441)
etc = ExtraTreesClassifier(criterion='entropy', max_depth=18, min_samples_split=0.01,
                     n_estimators=325)
est = [('rf', rf), ('log', log), ('xgb', xgb), ('etc', etc)]
stack = StackingClassifier(estimators=est,\
              final_estimator= xgb, cv=10, passthrough=False, verbose=1)
stack.fit(X_train, ytrain)
print(stack.estimators_, stack.final_estimator_, stack.stack_method_, sep='\n')

In [ ]:
# print(stack.score(X_test, ytest))
from sklearn.metrics import log_loss, accuracy_score
# from collections import Counter
# pred = stack.predict(X_test)
log_loss(ytest,stack.predict_proba(X_test))

# Preparing for Submission

In [ ]:
# test_data.info(verbose=True, show_counts=True)
cols = train_data_clean_1.columns
submission = pd.DataFrame()
submission = pd.concat([submission, test_data.uniqueid], axis=1)
test_data_imp = test_data.copy()
for col in test_data_imp.columns:
    if col in cols:
        test_data_imp.loc[test_data_imp[col].isnull(), col] = test_data_imp[col].mode()[0]
#         test_data_imp.loc[test_data_imp[col].isnull(), col] = floor(test_data_imp[col].median())
    else:
        test_data_imp.drop([col], axis=1,inplace=True)
print(test_data_imp.shape)
# test_data.info(verbose=True,show_counts=True)

In [ ]:
# Final_model =AdaBoostClassifier(DecisionTreeClassifier(),random_state=441, learning_rate=0.5, n_estimators=1)
# Final_model = RandomForestClassifier(criterion='entropy', max_depth=10,
#                        min_impurity_decrease=0.75, min_samples_split=0.5,
#                        n_estimators=250)
# Final_model = AdaBoostClassifier(base_estimator=RandomForestClassifier(criterion='entropy'),
#                    learning_rate=0.5, n_estimators=1, random_state=441)
# Final_model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
#               eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
#               interaction_constraints='', learning_rate=0.2, max_delta_step=0,
#               max_depth=2, min_child_weight=1,
#               monotone_constraints='()', n_estimators=100, n_jobs=-1,
#               num_parallel_tree=1, objective='multi:softprob', predictor='auto',
#               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
#               subsample=1, tree_method='exact', validate_parameters=1,
#               verbosity=0)
# Final_model = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=18,
#                        min_impurity_decrease=0, min_samples_split=0.01,
#                        n_estimators=255, warm_start=True)

Final_model = StackingClassifier(estimators=est,\
              final_estimator= xgb, cv=10, passthrough=False, verbose=1)
X = train_data_clean_1.iloc[:,:-1]
y = train_data_clean_1['health']
encoder = LabelEncoder()
encoder.fit(y)
yenc = encoder.transform(y)
Final_model.fit(X,yenc)
Final_model.score(X,yenc)

In [ ]:
predictions = Final_model.predict_proba(test_data_imp)
# predictions
for i in range(5):
    submission['p'+str(i+1)] = predictions[:,i]
print(submission.sample(10))

In [ ]:
submission.to_csv('/kaggle/working/submission.csv',index=False)